In [12]:
!pip install transformers
!pip install transformers-interpret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.8/793.8 KB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.1 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 68.0 MB/s eta 0:00:00m eta 0:00:01:010:01
  Using cached kiwisolver-1.4.4-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 KB 37.4 MB/s eta 0:00:00
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 71.4 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 62.5 MB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
  Attempting uninstall: ipython
    Found existing installation: ipython 8.13.2
    Uninstalling ipython-8.13.2:
      Successfully uninstalled ipython-8.13.2


In [2]:
import re
import porcodio

In [13]:
from transformers_interpret import SequenceClassificationExplainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

/home/funicello/.local/share/virtualenvs/ferrara-v9C_kj3N/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
pattern = r".*:.*"

with open("mlp_the_movie.txt", "rt") as file:
    lines = file.readlines()
    characters = dict()
    data = []
    for line in lines:
        if re.match(pattern,line):
            talker = line.split(":")[0].replace("'","").strip()
            phrase = line.split(":")[-1].strip()
            data.append([talker,phrase])
            if talker in characters:
                characters[talker] = characters[talker]+1
            else:
                characters[talker] = 1

In [5]:
for item in sorted(characters.items(), key=lambda x:x[1]):
    talker = item[0]
    if " and " in talker:
        if "," in talker:
            phrase = talker.replace(" and ",",")
            for p in phrase.split(","):
                p = p.strip()
                if p in characters: characters[p]= characters[p]+1
                else: characters[p] = 1
            del characters[talker]
            continue
        p1, p2 = talker.split(" and ")[0].strip(),talker.split(" and ")[1].strip()
        
        if p1 in characters: characters[p1] = characters[p1]+1
        else: characters[p1] = 1
        
        if p2 in characters: characters[p2] = characters[p2]+1
        else: characters[p2] = 1

        del characters[talker]
    if talker == "Mane Six":
        mane_six = ["Spike","Rainbow Dash","Pinkie Pie","Rarity","Applejack","Twilight Sparkle"]
        for horse in mane_six:
            characters[horse] += item[1]
        del characters[talker]
    if talker == "Mane Six except Twilight":
        mane_six_except = ["Spike","Rainbow Dash","Pinkie Pie","Rarity","Applejack"]
        for horse in mane_six_except:
            characters[horse] += item[1]
        del characters[talker]

In [15]:
model_name = "arpanghoshal/EmoRoBERTa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name,from_tf=True)
cls_explainer = SequenceClassificationExplainer(model,tokenizer)

2023-07-09 21:05:44.962763: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-09 21:05:45.160820: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-09 21:05:45.878614: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
All TF 2.0 model weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, y

In [16]:
for row in data:
    row.append(cls_explainer(row[1]))
    row.append(cls_explainer.id2label[cls_explainer.selected_index])

In [19]:
data

[['Spike',
  "'Scuse me! Dragon on the move! Important princess documents comin' through!",
  [('<s>', 0.0),
   ("'", 0.06435610805766359),
   ('Sc', -0.22770375569393894),
   ('use', -0.5999644206464442),
   ('me', 0.606842603389044),
   ('!', -0.24736440378662058),
   ('Dragon', 0.269717976839187),
   ('on', 0.012890020296907782),
   ('the', -0.03123258832527795),
   ('move', -0.16971939058700808),
   ('!', -0.07421695807458684),
   ('Important', -0.04511634874899131),
   ('princess', -0.07422696715136927),
   ('documents', -0.17527418508369655),
   ('com', -0.03771699376358879),
   ('in', -0.009957387332935906),
   ("'", -0.020265546656521694),
   ('through', -0.03927054269941892),
   ('!', -0.06789390866720933),
   ('', -0.007232518242059),
   ('</s>', 0.0)],
  'neutral'],
 ['S04E26 Unnamed Earth Mare #1',
  'Whoa! Everypony from Manehattan to Saddle Arabia is here!',
  [('<s>', 0.0),
   ('Who', 0.6682091521462572),
   ('a', 0.5359017806136077),
   ('!', 0.014091608953067141),
   (